In [ ]:
pip install llama-index-readers-database

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")
 

Langfuse client is authenticated and ready!


In [3]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
 
# Initialize LlamaIndex instrumentation
LlamaIndexInstrumentor().instrument()

In [4]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-4o-mini")

Settings.embed_model = embed_model
Settings.llm = llm

In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import sqlite3
from datetime import datetime

def create_database():
    # Connect to SQLite database (creates it if it doesn't exist)
    conn = sqlite3.connect('ecommerce.db')
    cursor = conn.cursor()

    # Create Products table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        description TEXT,
        category TEXT,
        price REAL,
        features TEXT,
        technical_specs TEXT
    )
    ''')

    # Create Customer Reviews table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS customer_reviews (
        review_id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id INTEGER,
        customer_name TEXT,
        rating INTEGER,
        review_text TEXT,
        review_date DATE,
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
    ''')

    # Sample product data
    products = [
        (
            'Ultra Comfort Ergonomic Chair',
            'Experience unparalleled comfort with our premium ergonomic office chair. Designed for professionals who spend long hours at their desk, this chair combines modern aesthetics with advanced ergonomic features.',
            'Office Furniture',
            299.99,
            'Adjustable lumbar support, 4D armrests, Memory foam cushioning, 360-degree swivel base, Height adjustment',
            'Weight capacity: 300lbs, Height range: 17-21 inches, Material: Mesh back with premium fabric seat, Assembly required'
        ),
        (
            'Smart Home Security Camera Pro',
            'Keep your home safe with our advanced AI-powered security camera. Features night vision, motion detection, and seamless integration with popular smart home platforms.',
            'Smart Home',
            159.99,
            'AI motion detection, 4K HDR recording, Two-way audio, Cloud storage, Weather-resistant',
            'Resolution: 4K (3840x2160), Field of view: 160°, Night vision range: 30ft, Wi-Fi: 2.4GHz & 5GHz, Battery life: 6 months'
        ),
        (
            'Organic Cotton Yoga Mat',
            'Connect with nature during your yoga practice using our eco-friendly cotton yoga mat. Made from 100% organic cotton using traditional weaving techniques.',
            'Fitness',
            89.99,
            'Non-slip surface, Natural materials, Machine washable, Extra thick padding, Carrying strap included',
            'Dimensions: 72" x 24", Thickness: 5mm, Material: 100% organic cotton, Care: Machine wash cold'
        )
    ]

    # Insert products
    cursor.executemany('''
    INSERT INTO products (name, description, category, price, features, technical_specs)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', products)

    # Sample review data
    reviews = [
        (
            1,  # product_id for the chair
            'Sarah Johnson',
            5,
            'This chair has completely transformed my work-from-home setup. The lumbar support is exceptional, and I no longer experience back pain after long hours of sitting. The assembly was straightforward, and the quality is outstanding.',
            '2024-01-15'
        ),
        (
            1,  # product_id for the chair
            'Michael Chen',
            4,
            'Great chair overall. The adjustable features are fantastic, though I wish the armrests had a bit more padding. Still, it\'s a significant upgrade from my previous office chair.',
            '2024-02-01'
        ),
        (
            2,  # product_id for the camera
            'Emily Rodriguez',
            5,
            'The image quality is incredible, especially at night. The AI features are really smart - it can distinguish between people, animals, and vehicles. Setup was a breeze with the app.',
            '2024-01-20'
        ),
        (
            3,  # product_id for the yoga mat
            'David Wilson',
            4,
            'Love the eco-friendly materials and the thickness is perfect for my yoga practice. The natural cotton provides excellent grip. Only giving 4 stars because it takes a while to dry after washing.',
            '2024-02-10'
        )
    ]

    # Insert reviews
    cursor.executemany('''
    INSERT INTO customer_reviews (product_id, customer_name, rating, review_text, review_date)
    VALUES (?, ?, ?, ?, ?)
    ''', reviews)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

def verify_database():
    """Function to verify the database was created correctly"""
    conn = sqlite3.connect('ecommerce.db')
    cursor = conn.cursor()

    print("Products in database:")
    cursor.execute("SELECT name, category, price FROM products")
    for row in cursor.fetchall():
        print(f"- {row[0]} ({row[1]}): ${row[2]}")

    print("\nReviews in database:")
    cursor.execute("""
        SELECT p.name, r.customer_name, r.rating, substr(r.review_text, 1, 50) || '...'
        FROM customer_reviews r
        JOIN products p ON r.product_id = p.product_id
    """)
    for row in cursor.fetchall():
        print(f"- {row[1]} rated {row[0]} {row[2]}/5 stars: {row[3]}")

    conn.close()


create_database()
print("Database created successfully!")
verify_database()

Database created successfully!
Products in database:
- Ultra Comfort Ergonomic Chair (Office Furniture): $299.99
- Smart Home Security Camera Pro (Smart Home): $159.99
- Organic Cotton Yoga Mat (Fitness): $89.99

Reviews in database:
- Sarah Johnson rated Ultra Comfort Ergonomic Chair 5/5 stars: This chair has completely transformed my work-from...
- Michael Chen rated Ultra Comfort Ergonomic Chair 4/5 stars: Great chair overall. The adjustable features are f...
- Emily Rodriguez rated Smart Home Security Camera Pro 5/5 stars: The image quality is incredible, especially at nig...
- David Wilson rated Organic Cotton Yoga Mat 4/5 stars: Love the eco-friendly materials and the thickness ...


In [6]:
from llama_index.readers.database import DatabaseReader

sqlite_uri = "sqlite:///ecommerce.db"

db = DatabaseReader(
    scheme="sqlite",
    uri=sqlite_uri
)

In [7]:
all_tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
all_tables = db.load_data(query=all_tables_query)
all_tables

[Document(id_='3dff4041-00cc-4b65-ae6a-81668779e4ea', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='name: products', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='9ffbce13-75c2-4d3a-98d7-ce7315b6a211', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='name: sqlite_sequence', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='c1388595-ba5d-460e-b21d-240a5405778b', embedding=None, metadata={}, excluded

In [8]:
documents = []
for table in all_tables:
    table_name = table.text.split(":")[1].strip()
    # print(table_name)
    query = f"SELECT * FROM {table_name}"
    table_data = db.load_data(query=query)
    documents.extend(table_data)
    

In [9]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-4o-mini")

Settings.embed_model = embed_model
Settings.llm = llm

In [10]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query("Show me eco fridenly exercise equipment")
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='One option for eco-friendly exercise equipment is the Organic Cotton Yoga Mat. It is made from 100% organic cotton and features a non-slip surface, making it suitable for yoga practices. Additionally, it is machine washable and has extra thick padding for comfort.', source_nodes=[NodeWithScore(node=TextNode(id_='e329b3f3-c9ba-49a0-b2b0-758f6a64f10f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='16967423-55dd-48f5-bbe3-f1180dad2f4f', node_type='4', metadata={}, hash='de75536cb48ee5eaf115fa7cdb6096efc8f37a7261f31c8f1626cd6fa1733113')}, metadata_template='{key}: {value}', metadata_separator='\n', text='product_id: 3, name: Organic Cotton Yoga Mat, description: Connect with nature during your yoga practice using our eco-friendly cotton yoga mat. Made from 100% organic cotton using traditional weaving techniques., category: Fitness, price: 89.99, feature

In [17]:
response = query_engine.query("Give me products that help with back pain")
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='One product that can help with back pain is the Ultra Comfort Ergonomic Chair. It features exceptional lumbar support, which is designed to alleviate discomfort during long hours of sitting. Additionally, it includes adjustable features and memory foam cushioning for enhanced comfort.', source_nodes=[NodeWithScore(node=TextNode(id_='42c72b50-f035-447d-9753-36bc0985c4fe', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a28ba425-ab96-4ae9-838f-e4650c64a849', node_type='4', metadata={}, hash='85335acd28154dedea6af65ec2bbe1220345c17c84ac3b237d27bbacc94f1bb9')}, metadata_template='{key}: {value}', metadata_separator='\n', text='review_id: 1, product_id: 1, customer_name: Sarah Johnson, rating: 5, review_text: This chair has completely transformed my work-from-home setup. The lumbar support is exceptional, and I no longer experience back pain after long hour

Hypothetical Document Embeddings for Reconstruct original query so that it will have keywords for better retrieval

In [11]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

hyde = HyDEQueryTransform(include_original=True)

hyde_query_engine = TransformQueryEngine(query_engine=query_engine, 
                                         query_transform=hyde)

response = hyde_query_engine.query("Give me products that help with back pain")
response

Response(response='The Ultra Comfort Ergonomic Chair is designed to provide exceptional lumbar support, making it a great option for alleviating back pain, especially for those who spend long hours sitting. Its features include adjustable lumbar support and memory foam cushioning, which contribute to comfort and support during extended use.', source_nodes=[NodeWithScore(node=TextNode(id_='c394a400-28a7-48cb-87a5-b7853b72f062', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2dead5ff-374d-4dc9-be94-a2893bc156fd', node_type='4', metadata={}, hash='90d39ffa4b81dd3db0d8011563c8c41814de49c161d8fd94178361694ff8fda6')}, metadata_template='{key}: {value}', metadata_separator='\n', text='product_id: 1, name: Ultra Comfort Ergonomic Chair, description: Experience unparalleled comfort with our premium ergonomic office chair. Designed for professionals who spend long hours at their 